In [1]:
!pip install --upgrade pip setuptools wheel


In [2]:
!pip install tpot


In [4]:
!pip install "setuptools<81"

In [5]:
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 4.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ipywidgets]3 [ipywidgets]widgets]


In [6]:
!pip install jupyterlab_widgets

In [9]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, RocCurveDisplay, auc
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_validate
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
import timeit

In [1]:
import pandas as pd

df = pd.read_csv(
    "new_data_train_Yelp_Fake_Review.csv",
    engine="python",
    sep="\t",# more tolerant
    on_bad_lines="skip"   # skip malformed rows
)

df.head()

,Unnamed: 0,reviewID,reviewerID,restaurantID,date,rating,reviewUsefulCount,reviewContent,flagged,name,...,reviewCount,firstCount,usefulCount,coolCount,funnyCount,complimentCount,tipCount,fanCount,restaurantRating,ReviewLength
0,6010,Z_wgGcI8_Txo87Wz7CxdrQ,cM9GIlk61Qh2thLyJyxNyA,N7juvW8YoFIj7sEBzVF8GQ,9/17/2010,5,0,as new resident chicago los angeles i mission ...,1,suzie n.,...,1,0,1,2,0,0,0,0,4.0,144
1,6566,NJfQb03MSstqYagt860Oaw,SVlSPBnjCzY2Bu9n8y2C5A,o54U2VkQama8FI30qDkWvw,1/31/2008,5,0,great place rarely wait fairly easy find parki...,0,Jacqui B.,...,12,2,19,7,6,4,0,0,4.0,46
2,21349,2uao3bK9iSV4fuMwuSYCiQ,5HYWhPS3ozYifieW0lWMUw,1QKqtC4vML3QhkrSzwR_tQ,3/31/2010,5,0,back bar makes feel though transported china s...,1,Mike A.,...,6,0,7,3,2,0,0,0,3.5,18
3,10370,6mVU2JfeSbQ-5RnPm-K6eA,dgKmhiTypR34XNPrMtQyMA,4wreIFjaPMppyEnzKHbozg,4/26/2010,5,0,hands one favorite tourist restaurants city th...,0,Jen K.,...,37,2,16,3,3,1,0,1,3.5,41
4,6011,e25NUAMnVf7-t8-vBO_drA,uB3n74XHDFUKwHxfK5lDvA,N7juvW8YoFIj7sEBzVF8GQ,9/7/2010,5,0,service impeccable staff helpful owner works h...,1,Joe K.,...,3,0,2,1,0,0,0,0,4.0,40


In [2]:
df = df[["reviewContent", "flagged"]]
df.head()

,reviewContent,flagged
0,as new resident chicago los angeles i mission ...,1
1,great place rarely wait fairly easy find parki...,0
2,back bar makes feel though transported china s...,1
3,hands one favorite tourist restaurants city th...,0
4,service impeccable staff helpful owner works h...,1


In [6]:
import pandas as pd
from tpot import TPOTClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score
from joblib import dump

# 1) Read train/test (tab-separated)
train_df = pd.read_csv("new_data_train_Yelp_Fake_Review.csv", sep="\t", engine="python")
test_df  = pd.read_csv("new_data_test_Yelp_Fake_Review.csv",  sep="\t", engine="python")

# 2) Keep only needed columns
train_df = train_df[["reviewContent", "flagged"]].dropna()
test_df  = test_df[["reviewContent", "flagged"]].dropna()

# 3) X/y
X_train = train_df["reviewContent"].astype(str)
y_train = train_df["flagged"].astype(int)

X_test  = test_df["reviewContent"].astype(str)
y_test  = test_df["flagged"].astype(int)

In [7]:
# Ensure labels are 0/1 ints
y_train = y_train.astype(int)
y_test  = y_test.astype(int)

print("X_train length:", len(X_train))
print("y_train length:", len(y_train))
print("X_test length :", len(X_test))
print("y_test length :", len(y_test))

X_train length: 9926
y_train length: 9926
X_test length : 2483
y_test length : 2483


In [10]:
# ===============================
# 4) Sanity checks (IMPORTANT)
# ===============================
print("Train size:", X_train.shape)
print("Test size :", X_test.shape)

print("Train labels:", np.unique(y_train, return_counts=True))
print("Test labels :", np.unique(y_test, return_counts=True))


Train size: (9926,)
Test size : (2483,)
Train labels: (array([0, 1]), array([4993, 4933]))
Test labels : (array([0, 1]), array([1212, 1271]))


In [12]:
import re
# ===============================
# 5) Text cleaning
# ===============================
def clean_text(s):
    s = s.lower()
    s = re.sub(r"http\S+|www\.\S+", " ", s)   # remove URLs
    s = re.sub(r"[^a-z\s]", " ", s)           # keep letters only
    s = re.sub(r"\s+", " ", s).strip()        # normalize spaces
    return s

X_train_clean = X_train.map(clean_text)
X_test_clean  = X_test.map(clean_text)


In [14]:
from tpot import TPOTClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report

# ===============================
# 6) TF-IDF (STRONG for fake reviews)
# ===============================
tfidf = TfidfVectorizer(
    analyzer="char_wb",       # 🔥 best for spam/fake reviews
    ngram_range=(3, 5),
    min_df=2,
    max_features=100000,
    sublinear_tf=True
)

# ===============================
# 7) TPOT (stable, server-safe)
# ===============================
tpot = TPOTClassifier(
    generations=5,            # increase later
    population_size=30,
    cv=3,
    random_state=42,
    verbose=2,
    n_jobs=-1,                # use all CPU cores
    client=None,              # ❌ disable dask
    scatter=False,
    max_time_mins=None
)


# ===============================
# 8) Pipeline: TF-IDF → TPOT
# ===============================
model = Pipeline([
    ("tfidf", tfidf),
    ("tpot", tpot),
])


# ===============================
# 9) Train
# ===============================
model.fit(X_train_clean, y_train)


# ===============================
# 10) Evaluate
# ===============================
y_pred = model.predict(X_test_clean)

print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("F1 (weighted):", f1_score(y_test, y_pred, average="weighted"))

print("\nClassification report:\n")
print(classification_report(y_test, y_pred, digits=4))


Generation:   0%|                                         | 0/5 [00:00<?, ?it/s]

StopIteration: 

In [ ]:
import pandas as pd
import numpy as np
import re

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report

from dask.distributed import Client, LocalCluster, get_client
from tpot import TPOTClassifier
from joblib import dump

# -------------------------------
# 0) Close any existing Dask client (important on notebooks)
# -------------------------------
try:
    c = get_client()
    c.close()
except Exception:
    pass

# -------------------------------
# 1) Read TRAIN / TEST (TSV)
# -------------------------------
train_df = pd.read_csv("new_data_train_Yelp_Fake_Review.csv", sep="\t", engine="python")
test_df  = pd.read_csv("new_data_test_Yelp_Fake_Review.csv",  sep="\t", engine="python")

# -------------------------------
# 2) Keep only needed columns
# -------------------------------
train_df = train_df[["reviewContent", "flagged"]].dropna().reset_index(drop=True)
test_df  = test_df[["reviewContent", "flagged"]].dropna().reset_index(drop=True)

X_train = train_df["reviewContent"].astype(str)
y_train = train_df["flagged"].astype(int)

X_test  = test_df["reviewContent"].astype(str)
y_test  = test_df["flagged"].astype(int)

print("Train labels:", np.unique(y_train, return_counts=True))
print("Test labels :", np.unique(y_test,  return_counts=True))

# -------------------------------
# 3) Text cleaning
# -------------------------------
def clean_text(s: str) -> str:
    s = str(s).lower()
    s = re.sub(r"http\S+|www\.\S+", " ", s)
    s = re.sub(r"[^a-z\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

X_train_clean = X_train.map(clean_text)
X_test_clean  = X_test.map(clean_text)

# -------------------------------
# 4) Start a clean Dask LocalCluster (parallel cores)
# -------------------------------
cluster = LocalCluster(
    n_workers=4,            # change to your CPU cores (e.g., 2/4/8)
    threads_per_worker=1,
    processes=True,
    dashboard_address=None  # avoids port 8787 warning
)
client = Client(cluster)

# -------------------------------
# 5) TF-IDF + TPOT
# -------------------------------
tfidf = TfidfVectorizer(
    analyzer="char_wb",
    ngram_range=(3, 5),
    min_df=2,
    max_features=100000,
    sublinear_tf=True
)

tpot = TPOTClassifier(
    generations=5,
    population_size=30,
    cv=3,
    random_state=42,
    verbose=2,
    n_jobs=1,          #  keep 1 when using Dask workers
    client=client,     # provide real workers
    scatter=True,
    max_time_mins=None
)

model = Pipeline([
    ("tfidf", tfidf),
    ("tpot", tpot),
])

# -------------------------------
# 6) Train + Evaluate
# -------------------------------
model.fit(X_train_clean, y_train)

y_pred = model.predict(X_test_clean)
print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("F1 (weighted):", f1_score(y_test, y_pred, average="weighted"))
print("\nReport:\n", classification_report(y_test, y_pred, digits=4))

# -------------------------------
# 7) Save model
# -------------------------------
dump(model, "tpot_yelp_fake_review_model.joblib")
print("\nSaved: tpot_yelp_fake_review_model.joblib")

# -------------------------------
# 8) Cleanup Dask
# -------------------------------
client.close()
cluster.close()



/home/michael/tpot_jupyter/miniconda3/envs/tpot_env/lib/python3.10/site-packages/stopit/__init__.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Train labels: (array([0, 1]), array([4993, 4933]))
Test labels : (array([0, 1]), array([1212, 1271]))


Generation:   0%|                                         | 0/5 [00:00<?, ?it/s]/home/michael/tpot_jupyter/miniconda3/envs/tpot_env/lib/python3.10/site-packages/stopit/__init__.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/home/michael/tpot_jupyter/miniconda3/envs/tpot_env/lib/python3.10/site-packages/stopit/__init__.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/home/michael/tpot_jupyter/miniconda3/envs/tpot_env/lib/python3.10/site-packages/stopit/__init__.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pyp

In [5]:
from dask.distributed import Client, LocalCluster
from tpot import TPOTClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score

# 0) Start a clean local Dask cluster (no dashboard => no 8787 warning)
cluster = LocalCluster(
    n_workers=4,            # change to 2 if server is small
    threads_per_worker=1,
    processes=True,
    dashboard_address=None
)
client = Client(cluster)

# 1) TPOT
tpot = TPOTClassifier(
    generations=5,
    population_size=30,
    cv=3,
    random_state=42,
    verbose=2,
    n_jobs=-1,
    client=client,          # ✅ provide real workers
    scatter=True,           # ✅ allow scatter
    max_time_mins=None      # ✅ remove warning
)

# 2) TFIDF + TPOT pipeline
model = Pipeline([
    ("tfidf", TfidfVectorizer(max_features=5000, ngram_range=(1, 2), stop_words="english")),
    ("tpot", tpot),
])

# 3) Train + evaluate
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 (weighted):", f1_score(y_test, y_pred, average="weighted"))

# 4) Cleanup (important on servers)
client.close()
cluster.close()


Generation:   0%|                                         | 0/5 [00:00<?, ?it/s]/home/michael/tpot_jupyter/miniconda3/envs/tpot_env/lib/python3.10/site-packages/stopit/__init__.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/home/michael/tpot_jupyter/miniconda3/envs/tpot_env/lib/python3.10/site-packages/stopit/__init__.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/home/michael/tpot_jupyter/miniconda3/envs/tpot_env/lib/python3.10/site-packages/stopit/__init__.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pyp

Accuracy: 0.6761981474023359
F1 (weighted): 0.6761444253824924


2025-12-16 09:04:32,388 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute('eval_objective_list-30984b9bd27171d2271fb4639b3f155d')" coro=<Worker.execute() done, defined at /home/michael/tpot_jupyter/miniconda3/envs/tpot_env/lib/python3.10/site-packages/distributed/worker_state_machine.py:3608>> ended with CancelledError
2025-12-16 09:04:36,388 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing


In [20]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(
    X.astype(np.float64),
    y.astype(int),
    train_size=0.8,
    test_size=0.2,
    random_state=30
)

tpot = TPOTClassifier(
    generations=10,
    population_size=100,
    cv=5,
    random_state=42,
    verbose=2,
    n_jobs=-1,
    client=None,     # ✅ disable dask client usage
    scatter=False    # ✅ do not scatter data to workers
)

tpot.fit(X_train, y_train)

best_model = tpot.fitted_pipeline_
y_pred = best_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

tpot.export("tpot_images_pipeline_gen_10_pop_100.py")


/home/michael/tpot_jupyter/miniconda3/envs/tpot_env/lib/python3.10/site-packages/tpot/tpot_estimator/estimator.py:458: UserWarning: Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.
  warnings.warn("Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.")
/home/michael/tpot_jupyter/miniconda3/envs/tpot_env/lib/python3.10/site-packages/distributed/node.py:188: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42701 instead
  warnings.warn(
Generation:   0%|                                        | 0/10 [00:00<?, ?it/s]

StopIteration: 

In [21]:
%%time
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tpot import TPOTClassifier

from dask.distributed import Client, LocalCluster

# 1) split
X_train, X_test, y_train, y_test = train_test_split(
    X.astype(np.float64),
    y.astype(int),
    train_size=0.8,
    test_size=0.2,
    random_state=30
)

# Sanity check: you must have >= 2 classes
print("Classes in train:", np.unique(y_train))

# 2) start a local dask cluster (no dashboard -> avoids 8787 warning)
cluster = LocalCluster(
    n_workers=4,              # adjust to your server cores
    threads_per_worker=1,
    processes=True,
    dashboard_address=None    # <-- avoids port 8787 issues
)
client = Client(cluster)

# 3) TPOT
tpot = TPOTClassifier(
    generations=10,
    population_size=100,
    cv=5,
    random_state=42,
    verbose=2,
    n_jobs=-1,
    client=client,
    scatter=True,
    max_time_mins=None        # <-- removes the warning about both being set
)

tpot.fit(X_train, y_train)

best_model = tpot.fitted_pipeline_
y_pred = best_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

tpot.export("tpot_images_pipeline_gen_10_pop_100.py")

# 4) cleanup
client.close()
cluster.close()


Classes in train: [0 1]


/home/michael/tpot_jupyter/miniconda3/envs/tpot_env/lib/python3.10/site-packages/distributed/node.py:188: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36259 instead
  warnings.warn(

Generation:   0%|                                        | 0/10 [00:00<?, ?it/s]/home/michael/tpot_jupyter/miniconda3/envs/tpot_env/lib/python3.10/site-packages/stopit/__init__.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/home/michael/tpot_jupyter/miniconda3/envs/tpot_env/lib/python3.10/site-packages/stopit/__init__.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrai

Accuracy: 0.948051948051948



/home/michael/tpot_jupyter/miniconda3/envs/tpot_env/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:110: UserWarning: Features [14] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/home/michael/tpot_jupyter/miniconda3/envs/tpot_env/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:111: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/home/michael/tpot_jupyter/miniconda3/envs/tpot_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


AttributeError: 'TPOTClassifier' object has no attribute 'export'

In [22]:
from joblib import dump

best_model = tpot.fitted_pipeline_
dump(best_model, "tpot_best_pipeline.joblib")

['tpot_best_pipeline.joblib']